In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from scipy import stats

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# Define the indices for the landmarks associated with the right and left eyes
right_eye_indices = [33, 246, 161, 160, 159, 158, 157, 173, 133, 7, 163, 144, 145, 153, 154, 155]
left_eye_indices = [362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382]

# Initialize VideoCapture with the video file
cap = cv2.VideoCapture("F:\\Download Folder\\Test_video.mkv")

# Lists to hold average coordinates
right_avg_x_values, right_avg_y_values, left_avg_x_values, left_avg_y_values = [], [], [], []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = face_mesh.process(frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            right_landmark_coords, left_landmark_coords = [], []

            for index in right_eye_indices:
                landmark = face_landmarks.landmark[index]
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                right_landmark_coords.extend([x, y])

            for index in left_eye_indices:
                landmark = face_landmarks.landmark[index]
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                left_landmark_coords.extend([x, y])

            right_avg_x = np.mean(right_landmark_coords[::2])
            right_avg_y = np.mean(right_landmark_coords[1::2])
            left_avg_x = np.mean(left_landmark_coords[::2])
            left_avg_y = np.mean(left_landmark_coords[1::2])

            right_avg_x_values.append(right_avg_x)
            right_avg_y_values.append(right_avg_y)
            left_avg_x_values.append(left_avg_x)
            left_avg_y_values.append(left_avg_y)

# Calculate metrics
metrics = {
    "Right Eye Avg X Std": np.std(right_avg_x_values),
    "Right Eye Avg Y Std": np.std(right_avg_y_values),
    "Left Eye Avg X Std": np.std(left_avg_x_values),
    "Left Eye Avg Y Std": np.std(left_avg_y_values),
    "Right Eye Avg X MAD": np.mean(np.abs(right_avg_x_values - np.mean(right_avg_x_values))),
    "Right Eye Avg Y MAD": np.mean(np.abs(right_avg_y_values - np.mean(right_avg_y_values))),
    "Left Eye Avg X MAD": np.mean(np.abs(left_avg_x_values - np.mean(left_avg_x_values))),
    "Left Eye Avg Y MAD": np.mean(np.abs(left_avg_y_values - np.mean(left_avg_y_values))),
    "Right Eye Avg X IQR": stats.iqr(right_avg_x_values),
    "Right Eye Avg Y IQR": stats.iqr(right_avg_y_values),
    "Left Eye Avg X IQR": stats.iqr(left_avg_x_values),
    "Left Eye Avg Y IQR": stats.iqr(left_avg_y_values)
}

# Convert to DataFrame
results_df = pd.DataFrame([metrics])

# Save the DataFrame to a CSV file
results_df.to_csv("eye_features.csv", index=False)

print("Eye feature analysis results saved to eye_features.csv")

# Clean-up resources
cap.release()


c:\Users\Windows 10\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Eye feature analysis results saved to eye_features.csv
